# Model Train

In [ ]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec

데이터는 올려주신 것에서 na만 떨구고 썼습니다

In [ ]:
data = pd.read_csv("data/NOUN_수정.txt").rename(columns={"Unnamed: 0":"ID"}).dropna()

In [ ]:
def modelTrain(cat):
    if 
    df = data[data.Category == cat]
    words = [str(x).split(" ") for x in politics.Nouns.values]
    model = Word2Vec(words, window=3, min_count=3, size=100, sg=1) # skip-gram 방식으로 모델링(random seed X)
    model.init_sims(replace=True) # 불필요한 메모리 deload
    filename = cat.replace("/","")
    model.save(f"./models/{filename}.model")

In [ ]:
np.vectorize(modelTrain)(data.Category.unique())